In [ ]:
import re
import nltk
# import cudf.pandas
# cudf.pandas.install()
import pandas as pd
from tqdm import tqdm 
tqdm.pandas()

lemmatizer = nltk.stem.WordNetLemmatizer()
from sklearn.feature_extraction.text import CountVectorizer

# Load Data

In [ ]:
df_cleantech_keywords = pd.read_json("/home/thiesen/Documents/Cleantech_Concepts/cleantech_keywords_similarity_015_co_occurrence_025_claim_fulltext.json")

In [ ]:
df_rel_on_science = pd.read_json('/mnt/hdd01/patentsview/Reliance on Science - Cleantech Patents/df_oaid_cleantech_lang_detect_yake_title_abstract_noun_chunks.json')

In [ ]:
df_rel_on_science['title_abstract'] = df_rel_on_science['title'].str.lower() + ' ' + df_rel_on_science['abstract'].str.lower()
df_rel_on_science['title_abstract_lemma'] = df_rel_on_science['title_abstract'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

# Build Document-Term Matrix

In [ ]:
ngrams = df_cleantech_keywords['keyword_yake_lemma'].tolist()
documents = df_rel_on_science['title_abstract_lemma'].tolist()

In [ ]:
def custom_tokenizer(text, ngrams):
    pattern = '|'.join(re.escape(phrase) for phrase in ngrams)
    tokens = []
    last_end = 0
    for match in re.finditer(pattern, text):
        tokens.extend(text[last_end:match.start()].split())
        tokens.append(match.group(0))
        last_end = match.end()
    tokens.extend(text[last_end:].split())
    return tokens

In [ ]:
def tokenizer_wrapper(text):
    return custom_tokenizer(text, ngrams)

In [ ]:
vectorizer = CountVectorizer(tokenizer=tokenizer_wrapper, vocabulary=ngrams)

In [ ]:
oaid_cleantech_dtm = vectorizer.fit_transform(documents)

In [ ]:
df_oaid_cleantech_dtm = pd.DataFrame(oaid_cleantech_dtm.toarray(), columns=vectorizer.get_feature_names_out())
df_oaid_cleantech_dtm.reset_index(drop=True, inplace=True)
df_rel_on_science.reset_index(drop=True, inplace=True)
df_oaid_cleantech_dtm.insert(0, 'oaid', df_rel_on_science['oaid'])

In [ ]:
df_oaid_cleantech_dtm.to_csv('/mnt/hdd01/Cleantech Network Analysis/df_oaid_cleantech_dtm.csv', index=False)